![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)


# **ZeroShotNerModel**

This notebook will cover the different parameters and usages of `ZeroShotNerModel` annotator. 

**📖 Learning Objectives:**

1. Understand how to use `ZeroShotNerModel`.

2. Become comfortable using the different parameters of the annotator.

3. Identify clinical entities on text without training data.


**🔗 Helpful Links:**

- Documentation : [ZeroShotNerModel](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#zeroshotnermodel)

- Python Docs : [ZeroShotNerModel](https://nlp.johnsnowlabs.com/licensed/api/python/reference/autosummary/sparknlp_jsl/annotator/ner/zero_shot_ner/index.html#sparknlp_jsl.annotator.ner.zero_shot_ner.ZeroShotNerModel)

- Scala Docs : [ZeroShotNerModel](https://nlp.johnsnowlabs.com/licensed/api/com/johnsnowlabs/finance/token_classification/ner/ZeroShotNerModel.html)

- For extended examples of usage, see the [Spark NLP Workshop repository](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/healthcare-nlp/).

## **📜 Background**


`ZeroShotNerModel` implements zero-shot named entity recognition by utilizing `BERT` transformer models.

As a zero-shot model, there is no need to train the model in a specific data  set, neither have the entities previously set.

## **🎬 Colab Setup**

In [ ]:
!pip install -q johnsnowlabs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.7/486.7 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 639.9/639.9 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.5 MB/s eta 0:00:00


In [ ]:
from johnsnowlabs import nlp


nlp.install(force_browser=True)

<IPython.core.display.Javascript object>

127.0.0.1 - - [11/May/2023 19:36:44] "GET /login?code=HtqWk4nQ1v5s1Byz1sVYXENYSLmO5R HTTP/1.1" 200 -


<IPython.core.display.Javascript object>

Licenses extracted successfully
📋 Stored John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👷 Setting up  John Snow Labs home in /root/.johnsnowlabs, this might take a few minutes.
🙆 JSL Home setup in /root/.johnsnowlabs
Installing /root/.johnsnowlabs/py_installs/spark_nlp_jsl-4.4.1-py3-none-any.whl to /usr/bin/python3
Installed 1 products:
💊 Spark-Healthcare==4.4.1 installed! ✅ Heal the planet with NLP! 


In [ ]:
from johnsnowlabs import nlp, medical
import pyspark.sql.functions as F

spark = nlp.start()

👌 Launched cpu optimized session with with: 🚀Spark-NLP==4.4.1, 💊Spark-Healthcare==4.4.1, running on ⚡ PySpark==3.1.2


## **🖨️ Input/Output Annotation Types**

- Input: `DOCUMENT`, `TOKEN`

- Output: `NAMED_ENTITY`

## **🔎 Parameters**


- `entityDefinitions`: A dictionary with definitions of the named entities. The keys of dictionary are the entity types and the values are lists of hypothesis templates.
- `predictionThreshold`: Minimal confidence score to consider the entity(Default: `0.01`)
- `ignoreEntitites`: A list of entities to be discarted from the output..


All the parameters can be set using the corresponding set method in camel case. For example, `.setMultiLabel()`.

### `relationalCategories`

In [ ]:
documentAssembler = nlp.DocumentAssembler().setInputCol("text").setOutputCol("document")

sentenceDetector = (
    nlp.SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")
)

tokenizer = nlp.Tokenizer().setInputCols(["sentence"]).setOutputCol("token")

zero_shot_ner = (
    medical.ZeroShotNerModel.pretrained(
        "zero_shot_ner_roberta", "en", "clinical/models"
    )
    .setEntityDefinitions(
        {
            "PROBLEM": [
                "What is the disease?",
                "What is his symptom?",
                "What is her disease?",
                "What is his disease?",
                "What is the problem?",
                "What does a patient suffer",
                "What was the reason that the patient is admitted to the clinic?",
            ],
            "DRUG": [
                "Which drug?",
                "Which is the drug?",
                "What is the drug?",
                "Which drug does he use?",
                "Which drug does she use?",
                "Which drug do I use?",
                "Which drug is prescribed for a symptom?",
            ],
            "ADMISSION_DATE": ["When did patient admitted to a clinic?"],
            "PATIENT_AGE": [
                "How old is the patient?",
                "What is the gae of the patient?",
            ],
        }
    )
    .setInputCols(["sentence", "token"])
    .setOutputCol("zero_shot_ner")
    .setPredictionThreshold(0.1)
)

ner_converter = (
    medical.NerConverterInternal()
    .setInputCols(["sentence", "token", "zero_shot_ner"])
    .setOutputCol("ner_chunk")
)
pipeline = nlp.Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        zero_shot_ner,
        ner_converter,
    ]
)

zero_shot_ner_model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

zero_shot_ner_roberta download started this may take some time.
Approximate size to download 438.9 MB
[OK!]


In [ ]:
zero_shot_ner.getClasses()

['DRUG', 'PATIENT_AGE', 'ADMISSION_DATE', 'PROBLEM']

In [ ]:
text_list = [
    "The doctor pescribed Majezik for my severe headache.",
    "The patient was admitted to the hospital for his colon cancer.",
    "27 years old patient was admitted to clinic on Sep 1st by Dr. X for a right-sided pleural effusion for thoracentesis.",
]

data = spark.createDataFrame(text_list, nlp.StringType()).toDF("text")

results = zero_shot_ner_model.transform(data)

In [ ]:
results.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|       zero_shot_ner|           ner_chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|The doctor pescri...|[{document, 0, 51...|[{document, 0, 51...|[{token, 0, 2, Th...|[{named_entity, 0...|[{chunk, 21, 27, ...|
|The patient was a...|[{document, 0, 61...|[{document, 0, 61...|[{token, 0, 2, Th...|[{named_entity, 0...|[{chunk, 49, 60, ...|
|27 years old pati...|[{document, 0, 11...|[{document, 0, 11...|[{token, 0, 1, 27...|[{named_entity, 0...|[{chunk, 0, 11, 2...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [ ]:
results.select(
    F.explode(
        F.arrays_zip(
            results.token.result,
            results.zero_shot_ner.result,
            results.zero_shot_ner.metadata,
            results.zero_shot_ner.begin,
            results.zero_shot_ner.end,
        )
    ).alias("cols")
).select(
    F.expr("cols['0']").alias("token"),
    F.expr("cols['1']").alias("ner_label"),
    F.expr("cols['2']['sentence']").alias("sentence"),
    F.expr("cols['3']").alias("begin"),
    F.expr("cols['4']").alias("end"),
    F.expr("cols['2']['confidence']").alias("confidence"),
).show(
    50, truncate=100
)

+-------------+----------------+--------+-----+---+----------+
|        token|       ner_label|sentence|begin|end|confidence|
+-------------+----------------+--------+-----+---+----------+
|          The|               O|       0|    0|  2|      null|
|       doctor|               O|       0|    4|  9|      null|
|    pescribed|               O|       0|   11| 19|      null|
|      Majezik|          B-DRUG|       0|   21| 27|0.64671576|
|          for|               O|       0|   29| 31|      null|
|           my|               O|       0|   33| 34|      null|
|       severe|       B-PROBLEM|       0|   36| 41| 0.5526346|
|     headache|       I-PROBLEM|       0|   43| 50| 0.5526346|
|            .|               O|       0|   51| 51|      null|
|          The|               O|       0|    0|  2|      null|
|      patient|               O|       0|    4| 10|      null|
|          was|               O|       0|   12| 14|      null|
|     admitted|               O|       0|   16| 23|    

### `predictionThreshold`

We can see that some confidece scores were not high, let's change the threshold.

In [ ]:
zero_shot_ner.setPredictionThreshold(0.8)

pipeline = nlp.Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        zero_shot_ner,
        ner_converter,
    ]
)

zero_shot_ner_model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))
results = zero_shot_ner_model.transform(data)

results.select(
    F.explode(
        F.arrays_zip(
            results.token.result,
            results.zero_shot_ner.result,
            results.zero_shot_ner.metadata,
            results.zero_shot_ner.begin,
            results.zero_shot_ner.end,
        )
    ).alias("cols")
).select(
    F.expr("cols['0']").alias("token"),
    F.expr("cols['1']").alias("ner_label"),
    F.expr("cols['2']['sentence']").alias("sentence"),
    F.expr("cols['3']").alias("begin"),
    F.expr("cols['4']").alias("end"),
    F.expr("cols['2']['confidence']").alias("confidence"),
).show(
    50, truncate=100
)

+-------------+----------------+--------+-----+---+----------+
|        token|       ner_label|sentence|begin|end|confidence|
+-------------+----------------+--------+-----+---+----------+
|          The|               O|       0|    0|  2|      null|
|       doctor|               O|       0|    4|  9|      null|
|    pescribed|               O|       0|   11| 19|      null|
|      Majezik|               O|       0|   21| 27|      null|
|          for|               O|       0|   29| 31|      null|
|           my|               O|       0|   33| 34|      null|
|       severe|               O|       0|   36| 41|      null|
|     headache|               O|       0|   43| 50|      null|
|            .|               O|       0|   51| 51|      null|
|          The|               O|       0|    0|  2|      null|
|      patient|               O|       0|    4| 10|      null|
|          was|               O|       0|   12| 14|      null|
|     admitted|               O|       0|   16| 23|    

### `ignoreEntities`

In [ ]:
zero_shot_ner.setPredictionThreshold(0.45).setIgnoreEntities(["PATIENT_AGE"])

pipeline = nlp.Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        zero_shot_ner,
        ner_converter,
    ]
)

zero_shot_ner_model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))
results = zero_shot_ner_model.transform(data)

results.select(
    F.explode(
        F.arrays_zip(
            results.token.result,
            results.zero_shot_ner.result,
            results.zero_shot_ner.metadata,
            results.zero_shot_ner.begin,
            results.zero_shot_ner.end,
        )
    ).alias("cols")
).select(
    F.expr("cols['0']").alias("token"),
    F.expr("cols['1']").alias("ner_label"),
    F.expr("cols['2']['sentence']").alias("sentence"),
    F.expr("cols['3']").alias("begin"),
    F.expr("cols['4']").alias("end"),
    F.expr("cols['2']['confidence']").alias("confidence"),
).show(
    50, truncate=100
)

+-------------+----------------+--------+-----+---+----------+
|        token|       ner_label|sentence|begin|end|confidence|
+-------------+----------------+--------+-----+---+----------+
|          The|               O|       0|    0|  2|      null|
|       doctor|               O|       0|    4|  9|      null|
|    pescribed|               O|       0|   11| 19|      null|
|      Majezik|          B-DRUG|       0|   21| 27|0.64671576|
|          for|               O|       0|   29| 31|      null|
|           my|               O|       0|   33| 34|      null|
|       severe|       B-PROBLEM|       0|   36| 41| 0.5526346|
|     headache|       I-PROBLEM|       0|   43| 50| 0.5526346|
|            .|               O|       0|   51| 51|      null|
|          The|               O|       0|    0|  2|      null|
|      patient|               O|       0|    4| 10|      null|
|          was|               O|       0|   12| 14|      null|
|     admitted|               O|       0|   16| 23|    

## Fast inference with [LightPipelines](https://nlp.johnsnowlabs.com/docs/en/concepts#using-spark-nlps-lightpipeline)

We can use Spark NLP's `LightPipeline` to run fast inference directly on text (or list of text) instead of using spark data frames. 

Let's check how to do that.

In [ ]:
lp = nlp.LightPipeline(zero_shot_ner_model)

result = lp.annotate("27 years old patient was admitted to clinic on Sep 1st by Dr. X for a right-sided pleural effusion for thoracentesis.")
result

{'zero_shot_ner': ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ADMISSION_DATE',
  'I-ADMISSION_DATE',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PROBLEM',
  'I-PROBLEM',
  'I-PROBLEM',
  'I-PROBLEM',
  'I-PROBLEM',
  'I-PROBLEM',
  'O'],
 'document': ['27 years old patient was admitted to clinic on Sep 1st by Dr. X for a right-sided pleural effusion for thoracentesis.'],
 'ner_chunk': ['Sep 1st', 'a right-sided pleural effusion for thoracentesis'],
 'token': ['27',
  'years',
  'old',
  'patient',
  'was',
  'admitted',
  'to',
  'clinic',
  'on',
  'Sep',
  '1st',
  'by',
  'Dr',
  '.',
  'X',
  'for',
  'a',
  'right-sided',
  'pleural',
  'effusion',
  'for',
  'thoracentesis',
  '.'],
 'sentence': ['27 years old patient was admitted to clinic on Sep 1st by Dr. X for a right-sided pleural effusion for thoracentesis.']}

In [ ]:
for token, ner_label in zip(result["token"], result["zero_shot_ner"]):
  print(f"{token} => {ner_label}")

27 => O
years => O
old => O
patient => O
was => O
admitted => O
to => O
clinic => O
on => O
Sep => B-ADMISSION_DATE
1st => I-ADMISSION_DATE
by => O
Dr => O
. => O
X => O
for => O
a => B-PROBLEM
right-sided => I-PROBLEM
pleural => I-PROBLEM
effusion => I-PROBLEM
for => I-PROBLEM
thoracentesis => I-PROBLEM
. => O
